# EntropiaAI | Technical Assessment

## Phase 1. Setup

In [1]:
!python --version
!uv --version
#!uv pip list

Python 3.12.4
uv 0.10.2 (a788db7e5 2026-02-10)


In [2]:
from dotenv import load_dotenv
from pathlib import Path
import os

load_dotenv(Path.cwd() / ".env", override=True)

print("API Key loaded:", bool(os.getenv("GEMINI_API_KEY") or os.getenv("GOOGLE_API_KEY")))
print("Model:", os.getenv("GEMINI_MODEL", "gemini-2.0-flash"))


API Key loaded: True
Model: gemini-2.5-flash-lite


In [3]:
from src.llm import getChatModel
from src.router import decideFile
from src.retriever import retrieveAndAnswer
from src.stylist import styleAnswer, StyledAnswers
from src.workflow import questionAgent
import json

## Phase 2. The Core Logic

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

llm = getChatModel()
demo_messages = [
    SystemMessage(content="Eres un asistente. Responde solo: OK."),
    HumanMessage(content="Hola")
]
resp = llm.invoke(demo_messages)
print("Response:", resp.content)


## Phase 3. The Workflow

In [ ]:
# Router

tests_router = [
    "¿Qué dice el cookbook sobre el uso de etiquetas XML para GPT-4.1?",
    "¿Top 5 estados más calientes en agosto de 2021?",
    "¿Cuántas razas de maíz son nativas en México?",
    "¿Cuál es el clima en Marte?" 
]

for t in tests_router:
    d = decideFile(llm, t)
    print(f"Q: {t}\n -> file: {d.filename} | reason: {d.reason}\n")


In [ ]:
# Retriever

q = "¿Cuáles fueron las 3 temperaturas mínimas en enero de 2020 en Mexico?"
decision = decideFile(llm, q)
print("Selected:", decision.filename)

original_answer = retrieveAndAnswer(llm, decision.filename, q)
print(original_answer)



In [ ]:
# Stylist

styled = styleAnswer(llm, original_answer)

print("Output")
print(json.dumps(styled, ensure_ascii=False, indent=2))


In [ ]:
# Workflow

q = "¿Top 5 estados más calientes en agosto de 2021?"
out = questionAgent(q, model_name="gemini-2.5-flash-lite", include_trace=True)

print(json.dumps(out, ensure_ascii=False, indent=2))


In [ ]:
# Workflow (Imagen)

q = "¿Cuántas razas de maíz son nativas en México?"
out = questionAgent(q, model_name="gemini-2.5-flash-lite", include_trace=True)

print(json.dumps(out, ensure_ascii=False, indent=2))


## Test

In [5]:
tests = [
    # "¿Qué dice el cookbook sobre el uso de etiquetas XML para GPT-4.1?",
    "¿Algún consejo para promptear a GPT-5?",
    # "¿Top 5 estados más calientes en agosto de 2021?",
    # "¿Cuáles fueron las 3 temperaturas mínimas en diciembre de 2025? ¿En dónde?",
    # "¿Cuántas razas de maíz son nativas en México?",
    "¿Por qué el huitlacoche es superior en nutrientes al maíz tradicional?",
]

for i, q in enumerate(tests, 1):
    print(f"\nTEST {i}: {q}")

    out = questionAgent(q, model_name="gemini-2.5-flash", include_trace=True)

    trace = out.get("_trace", {})
    print("Selected file:", trace.get("selected_file"))
    print("Router reason:", trace.get("router_reason"))

    print("\nOriginal answer:\n", out.get("original_answer"))
    print("\nDry answer:\n", out.get("dry_answer"))
    print("\nFunny answer:\n", out.get("funny_answer"))


TEST 1: ¿Algún consejo para promptear a GPT-5?
Selected file: GPT-41_PromptingGuide.txt
Router reason: La pregunta busca consejos para promptear a un modelo GPT. Aunque menciona GPT-5, la guía de prompting para GPT-4.1 es el recurso más relevante y cercano disponible para ofrecer consejos sobre cómo interactuar con modelos de lenguaje de la serie GPT.

Original answer:
 No tengo información suficiente en el archivo proporcionado.

Dry answer:
 Información insuficiente en el archivo.

Funny answer:
 Mi sistema está buscando, pero este archivo está más vacío que mi nevera un lunes por la mañana. ¡No hay información suficiente!

TEST 2: ¿Por qué el huitlacoche es superior en nutrientes al maíz tradicional?
Selected file: maiz_info.jpg
Router reason: La pregunta es sobre el maíz y el huitlacoche. El archivo 'maiz_info.jpg' es una imagen que podría contener información relevante sobre el maíz, incluyendo comparaciones nutricionales o datos sobre el huitlacoche.

Original answer:
 No tengo 

## Reflection

### *Tokens & Cost* <br>
Por cada pregunta que se le hacia al agente, se consumian 3 llamadas al modelo (1 para elegir el archivo, 1 para responder con contexto y 1 para reescribir en JSON con los 3 tonos) 

Lo optimizaria, en un caso real solo añadir el estilo en caso de que sea requisito porque no aporta valor a la informacion, esencialmente es la misma pero de otra forma. Y para el caso de las imagenes, tal vez usar OCR (unicamente si se que las imagenes conservan mas texto y no se requiere encontrarle sentido a lo que se ve) y ese texto mejor pasarlo al modelo ya que por lo general son mas costosas las llamadas teniendo como entrada imagenes que texto.

### *Latency* <br>
Considero que las respuestas que da son un tiempo aceptable ya que por pregunta, se tardo alrededor de 8-9 segundos ya dando las 3 formas de estilo.

### *Evaluation* <br>
Automatizar pruebas (Con el costo que eso implica por las llamadas) para contar casos correctos de asignacion de archivo, de saber cuando el contexto no contiene la info y para preguntas con el CSV realizar ejemplos usando pandas. 

### *Escalability* <br>
Para tener unicamente 3 archivos de los cuales elegir funciona, pero escalar a mas tipos de archivos o diferentes archivos con la misma extension representa un problema para el LLM recordar este catalogo.

### *Autonomy* <br>
Ventajas: la rapidez en el desarrollo conservando el desempeño pero mayores costos
Contras: por lo regular suele ser mas barato y personalizable codificar, pero con la curva de aprendizaje que esto implica

### *Personal Retrospective* <br>
La evaluacion fue demasiado completa por abarcar las etapas que considero fundamentales en un sistema de preguntas y respuestas, con su nivel de complejidad para poder resolver algunos aspectos que no estaban muy frescos o en completo desconocimiento pero para la prueba el investigar e ingeniar la forma de resolverlo me parecio interesante. 
Si me veo en un futuro creando soluciones de este tipo por mi gusto e interes por aprender y especializarme en temas relacionados con la inteligencia artificial. Siento que un contexto tan cercado a la realidad obtenido en esta prueba refuerza mi decision de seguir por este camino y los retos que esto implique. 